In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import glob
import os
import polars as pl
# import duckdb as dd
from tqdm import tqdm
import matplotlib.pyplot as plt
import cv2
from pydicom import dcmread
import warnings
from sklearn.preprocessing import LabelEncoder
import pickle
import gc
import ctypes
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split

2024-08-12 04:35:23.579506: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-12 04:35:23.579660: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-12 04:35:23.748743: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
model = keras.models.\
load_model("/kaggle/input/keras_base_lnfn_l1_l2/keras/default/1/keras_base_left_neural_foraminal_narrowing_l1_l2.h5")

In [3]:
submission = pd.read_csv('/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/sample_submission.csv')
submission['row_id'] = 'samples'
#submission

In [4]:
Test = True
config = {}

if Test:
    config['root_file_path'] = '/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/train_images'
    config['start'] = 10
    config['end'] = 110
    studies = os.listdir(config['root_file_path'])[config['start']:config['end']]
    test_dict = {}
    image_files = []

    for study in studies:
        for dirname, _, filenames in os.walk(config['root_file_path']+'/'+study):
            for filename in filenames:
                test_dict[os.path.join(dirname, filename).split('/')[-3]] = image_files
                image_files.append(os.path.join(dirname, filename))
else:
    config['root_file_path'] = '/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/test_images/'
    test_dict = {}
    image_files = []

    for dirname, _, filenames in os.walk(config['root_file_path']):
        for filename in filenames:
            test_dict[os.path.join(dirname, filename).split('/')[-3]] = image_files
            image_files.append(os.path.join(dirname, filename))

In [ ]:
for key,value in test_dict.items():
    print("there are {1} files under {0}".format(key,len(value)))

In [ ]:
for st in tqdm(test_dict.keys()):
    print(st)

In [9]:
"""for file in test_dict['2532413137']:
    print(len(file))"""

for i in range(np.array(test_dict['2532413137']).shape[0]):
    print(np.array(test_dict['2532413137'])[i])
    if i == 10:
        break

/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/train_images/2532413137/3557372893/12.dcm
/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/train_images/2532413137/3557372893/18.dcm
/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/train_images/2532413137/3557372893/9.dcm
/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/train_images/2532413137/3557372893/22.dcm
/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/train_images/2532413137/3557372893/25.dcm
/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/train_images/2532413137/3557372893/39.dcm
/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/train_images/2532413137/3557372893/45.dcm
/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/train_images/2532413137/3557372893/14.dcm
/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/train_images/2532413137/3557372893/11.dcm
/kaggle/input/rsna-2024-lumbar-spine-d

In [47]:
feature_resized = []

for file in tqdm(test_dict['2532413137']):
    dicom_ds = dcmread(file)
    img_array = dicom_ds.pixel_array
    feature_resized.append(np.resize(np.mean(img_array.T, axis=0),(128,)))

100%|██████████| 7586/7586 [02:10<00:00, 57.98it/s] 


In [52]:
np.array(feature_resized).shape

(7586, 128)

In [54]:
feature_resized[0]

array([  5.715625,  31.965625,  31.821875,  32.4375  ,  32.21875 ,
        30.084375,  31.084375,  30.821875,  30.01875 ,  30.759375,
        31.58125 ,  31.559375,  31.803125,  31.478125,  32.6875  ,
        32.7875  ,  32.671875,  32.65625 ,  33.540625,  33.903125,
        34.0625  ,  34.921875,  36.5625  ,  37.378125,  36.896875,
        36.4125  ,  38.334375,  42.078125,  44.640625,  44.7875  ,
        47.0375  ,  50.70625 ,  52.275   ,  54.609375,  60.14375 ,
        63.9375  ,  68.053125,  73.15    ,  78.05    ,  81.971875,
        85.934375,  89.1375  ,  91.296875,  91.128125,  90.859375,
        91.975   ,  95.059375,  97.03125 ,  98.24375 , 100.43125 ,
       101.609375, 100.41875 , 100.284375,  99.871875,  97.36875 ,
        97.84375 , 100.865625, 102.76875 , 103.23125 , 102.896875,
       105.45625 , 103.434375, 101.55    , 104.7875  , 107.134375,
       104.865625, 105.1375  , 108.1625  , 107.85    , 110.21875 ,
       116.88125 , 122.25625 , 130.678125, 138.203125, 140.206

In [18]:
dicom_ds = dcmread('/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/train_images/2532413137/3557372893/12.dcm')
img_array = dicom_ds.pixel_array
test = np.resize(np.mean(img_array.T, axis=0),(128,))

In [36]:
type(test)

numpy.ndarray

In [60]:
def get_feature_array(img_file_list):
    #print(type(img_file_list))
    #feature_resized = []
    #for file in img_file_list:
    pbar.update(1)
    dicom_ds = dcmread(img_file_list)
    img_array = dicom_ds.pixel_array
    #feature_resized.append(np.resize(np.mean(img_array.T, axis=0),(128,)))
    #return feature_resized
    return np.resize(np.mean(img_array.T, axis=0),(128,))

vfunc = np.vectorize(get_feature_array, otypes=[object])

In [ ]:
"""import time
start = time.time()
print(start)"""
rows = {}
global pbar 
pbar = 0
for st in test_dict.keys():
    # print(st)
    with tqdm(total=len(test_dict[st])) as pbar:
        final_feature_list = vfunc(test_dict[st]).tolist()
    #final_feature_list = np.apply_along_axis(get_feature_array, 0, test_dict[st])
    final = np.array(final_feature_list)
    y_proba = model.predict(final)
    rows[st] = np.mean(y_proba, axis=0)
    
"""end = time.time()
print(end)
print("Took %f ms" % ((end - start) * 1000.0))"""

100%|██████████| 7586/7586 [02:03<00:00, 61.63it/s] 


238/238 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step


 27%|██▋       | 2018/7586 [00:33<01:28, 62.91it/s] 

In [56]:
#final_feature_list[0]
#model.predict(final.values.astype(np.float32))
list_1 = final_feature_list.tolist()

In [59]:
y_proba = model.predict(np.array(list_1))
#np.array(list_1).shape
#list_1

238/238 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step


In [44]:
final_feature_list[0].shape

(128,)

In [46]:
final_feature_list_tensor = tf.convert_to_tensor(final, dtype=tf.float64)

ValueError: setting an array element with a sequence.

In [ ]:
rows = {}
# weight_dict = {'normal_mild':1, 'moderate':2, 'severe':4}
conditions = ['spinal_canal_stenosis', 'neural_foraminal_narrowing', 'subarticular_stenosis']
sides = ['left', 'right']
vertebrae_levels = ['l1_l2', 'l2_l3', 'l3_l4', 'l4_l5', 'l5_s1']
severity_levels = ['normal_mild', 'moderate', 'severe']

for c in conditions:
    for v in vertebrae_levels:
        if c != 'spinal_canal_stenosis':
            for s in sides:
                for st in test_dict.keys():
                    if s+'_'+c+'_'+v == 'left_neural_foraminal_narrowing_l1_l2':
                        #print("going for model ", st+'_'+s+'_'+c+'_'+v)
                        #features = []
                        feature_resized = []
                        for file in tqdm(test_dict[st]):
                            dicom_ds = dcmread(file)
                            img_array = dicom_ds.pixel_array
                            feature_resized.append(np.resize(np.mean(img_array.T, axis=0),(128,)))
                        final = np.array(feature_resized)
                        y_proba = model.predict(final)
                        rows[st+'_'+s+'_'+c+'_'+v] = np.mean(y_proba, axis=0)
                    else:
                        #print(st+'_'+s+'_'+c+'_'+v)
                        rows[st+'_'+s+'_'+c+'_'+v] = np.array([0.333333 * 2, 0.333333 * 2, 0.333333 * 2])
        else:
            for st in test_dict.keys():
                #print(st+'_'+c+'_'+v)
                rows[st+'_'+c+'_'+v] = np.array([0.333333 * 2, 0.333333 * 2, 0.333333 * 2])

In [ ]:
for row_id, feature in tqdm(rows.items()):
    feature_set_reshaped = feature.reshape(1, -1)
    predictions = np.ascontiguousarray(feature_set_reshaped/2)
    df = pd.DataFrame(predictions, columns=severity_levels)
    df.insert(loc=0, column='row_id', value=row_id)
    submission = pd.concat([submission,df]).reset_index(drop=True)
    
i = submission[(submission.row_id == 'samples')].index
submission = submission.drop(i).reset_index(drop=True)

In [ ]:
submission.shape